In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#for prob 5
import math

#for prob 6
from sklearn.svm import SVC
from sklearn import svm
from copy import deepcopy

In [3]:
#problem 5 - evaluate P(bad)
tie_weight = .5

def pbad_n(K,eps,n):
    log_prob_component = n*np.log(eps) + (K-n)*np.log(1.-eps)
    log_fact_component = np.log(math.factorial(K)) - np.log((math.factorial(n)*math.factorial(K-n))) 
    return np.exp(log_prob_component + log_fact_component)

def pbad(K,eps):
    pout = 0.
    if(K%2 == 0):
        sumstart = int((K/2) +1)
        pout += tie_weight*pbad_n(K,eps,int(K/2))
    else:
        sumstart = int((K+1)/2)
    
    for nwrong in range(sumstart,K+1):
        currcontrib = pbad_n(K,eps,nwrong)
        pout += currcontrib
        
#         print("nwrong = " + str(nwrong) + "; contribution = " + str(currcontrib))
    
    return pout

for K in [5,10,20]:
    for eps in [.1,.2,.4]:
        p_exa = pbad(K,eps)
        print("\item $K = " + str(K) + ", \eps = " + str(eps) + "; P_{bad} = " + str(p_exa) +"$")
        
        

\item $K = 5, \eps = 0.1; P_{bad} = 0.00856$
\item $K = 5, \eps = 0.2; P_{bad} = 0.05792$
\item $K = 5, \eps = 0.4; P_{bad} = 0.31744$
\item $K = 10, \eps = 0.1; P_{bad} = 0.00089092$
\item $K = 10, \eps = 0.2; P_{bad} = 0.01958144$
\item $K = 10, \eps = 0.4; P_{bad} = 0.26656768$
\item $K = 20, \eps = 0.1; P_{bad} = 3.92988232713e-06$
\item $K = 20, \eps = 0.2; P_{bad} = 0.00157912054917$
\item $K = 20, \eps = 0.4; P_{bad} = 0.186092021415$


In [4]:
#problem 6 - SVM and margin separators

points = [[1.5,6],[2,1],[3,4],[4,4],[3.5,2],[4.5,4.5]]
classes = [1,1,1,0,0,0]
Xi = np.asarray(points)
yi = classes
y = deepcopy(yi)
print(Xi.shape)
print(len(yi))

(6, 2)
6


In [5]:
#for my input line, give distance of all points

def dist_to_line(point, m_in, b_in):
    #based on a formula for wikipedia
    #uses eqn of line ax + by + c = 0; can set a = m_in, b = -1, c = b_in
    a = m_in
    b = -1.
    c = b_in
    
    numer = abs(a*point[0] + b*point[1] + c)
    return numer/(np.sqrt(a**2. + b**2.))
    
def closest_dist_to_line(points, m_in, b_in, return_i = False):
    best_dist = dist_to_line(points[0], m_in, b_in)
    best_i = 0
    for i in range(1,len(points)):
        curr_dist = dist_to_line(points[i],m_in,b_in)
        if(curr_dist < best_dist):
            best_dist = curr_dist
            best_i = i
    if(return_i):
        return (best_dist, best_i)
    else:
        return best_dist


def evaluate_guess_line(points_in, guess_m, guess_b):
    print(closest_dist_to_line(points_in, guess_m, guess_b, return_i = True))
    
evaluate_guess_line(Xi, 4.,-10.)

(0.48507125007266594, 2)


In [6]:
#6b 
#just setup - see near bottom of notebook for plots and distance
linsvm = SVC(C=100.,kernel = 'linear')
linsvm.fit(Xi,yi)

In [7]:
# dotprods = linsvm.decision_function(Xi)

def eval_lin_svm(linsvm, Xi):
    w = linsvm.coef_[0]
    lin_guess_slope = -(w[0]+10**(-10.)) / (w[1]+10.**(-10.))
    lin_guess_int =  -(linsvm.intercept_[0]) / (w[1]+ 10.**(-10.))

    evaluate_guess_line(Xi, lin_guess_slope, lin_guess_int)
    
# eval_lin_svm(linsvm,Xi)
# dotprods

(0.48502356866976165, 3)


array([ 4.99941891,  1.49909204,  0.99985473, -0.99970945, -1.00014527,
       -1.74943707])

In [10]:
Xc = deepcopy(Xi)
Xc[4][0] = 5

array([[ 1.5,  6. ],
       [ 2. ,  1. ],
       [ 3. ,  4. ],
       [ 4. ,  4. ],
       [ 5. ,  2. ],
       [ 4.5,  4.5]])

In [11]:
#6d setup
X = deepcopy(Xi)
# X[4][0] = 3.5
y = deepcopy(yi)
y[5] = 1.

In [12]:
def plot_contours_nicely(Xin, kernel, C, show = True, savefolder = None, yin = yi, retsvm = False, polydeg = 4):
    xposns = []
    yposns = []
    for i in range(0,len(yin)):
        xposns.append(Xin[i][0])
        yposns.append(Xin[i][1])
    xposns = np.asarray(xposns)
    yposns = np.asarray(yposns)


    x_min, x_max = min(xposns)-1.,max(xposns)+1.
    y_min, y_max = min(yposns)-1.,max(yposns)+1.

    xrange = np.arange(x_min,x_max,.01)
    yrange = np.arange(y_min,y_max,.01)

    xx,yy = np.meshgrid(xrange,yrange)

    #'poly','rbf'
    # .1,1.,10.,100.

    currsvm = SVC(C=C, kernel=kernel,degree=polydeg)
    currsvm.fit(Xin,yin)
    curr_dotprods = currsvm.decision_function(Xin)


    plt.clf()
    preds = currsvm.predict(np.c_[xx.ravel(),yy.ravel()])
    preds = preds.reshape(xx.shape)

    xrange2 = np.arange(x_min,x_max,.1)
    yrange2 = np.arange(y_min,y_max,.1)

    dists = currsvm.decision_function(np.c_[xx.ravel(),yy.ravel()])
    dists = dists.reshape(xx.shape)

    plt.contourf(xx,yy,preds,alpha=.5)
    plt.contour(xx,yy,dists,levels=[-1,0.,1.],linestyles = ('dashed','solid','dashed'), colors = ('k','k','k'))
    
    
    marker0 = 'ko'
    marker1 = 'ks'
    xplots = [[],[]]
    yplots = [[],[]]
    for i in range(0,len(yin)):
        ind = 1
        if(yin[i] == yin[0]):
            ind = 0
        xplots[ind].append(xposns[i])
        yplots[ind].append(yposns[i])
    
    plt.plot(xplots[0],yplots[0],marker0)
    plt.plot(xplots[1],yplots[1],marker1)
#     plt.plot(xposns[0:3],yposns[0:3],'ko')
#     plt.plot(xposns[5],yposns[5],'ko')
#     plt.plot(xposns[3:5],yposns[3:5], 'k*')

    plt.title("Decision Boundary: Kernel = " + str(kernel) + "; C = " + str(C))

    if(not savefolder == None):
        plt.savefig(savefolder + '/' + 'boundary_' + str(kernel) + "_C_" + str(C/1000)[2:])
    if(show):
        plt.show()

#         plot_svm_margins(currsvm,X,y,"Margin Separator for " + str(kernel) + " kernel, C = " + str(C))
#     print(curr_dotprods)
    print(currsvm.score(Xin,yin))
    if(retsvm):
        return currsvm

In [13]:
def plot_contours_all(Xs,kernels,Cs,show=True,savefolder=None,y=yi,retsvms=False,polydeg=4):
    svms = []
    for X in Xs:
        for kernel in kernels:
            for C in Cs:
                if(retsvms):
                    currsvm = plot_contours_nicely(X,kernel,C,show,savefolder,y,retsvms,polydeg)
                    svms.append(currsvm)
                else:
                     plot_contours_nicely(X,kernel,C,show,savefolder,y,retsvms,polydeg)
                        
    if(retsvms):
        return svms

In [14]:
#6b and 6c plot
sixab_svms = plot_contours_all([Xi,Xc],['linear'],[100.],True,None,yi,True)

for svm_exa in sixab_svms:
    eval_lin_svm(svm_exa,Xi)
#note - evaluation function doesn't work for vertical lines, like the separator for part c
#visually and logically, the separator is a vertical line and ought to be halfway between x = 4 and x = 3
#so the distance to the closest point(s) will be .5

1.0
1.0
(0.48502356866976165, 3)
(2.7500076295320281e-10, 4)


In [15]:
#6d plot
sixd_svms = plot_contours_all([Xi],['linear'],[.1,1.,10.,100.],True,'6d_plots',y,True)

#print closest point + its distance for C = 100 svm
eval_lin_svm(sixd_svms[3],Xi)

0.666666666667
0.5
0.833333333333
0.833333333333
(0.31963073414732346, 4)


(0.3196307343208542, 4)


In [16]:
#6e plots - poly
deg = 4
poly_svms = plot_contours_all([X],['poly'],[.1,1.,10.,100.],True,'6e_poly_plots',y,True,deg)

#gives perfect score on training set for C = 100, 5/6 for lower C ( = more tolerance of error)

0.833333333333
0.833333333333
0.833333333333
1.0


In [18]:
#6e plots - rbf

rbf_svms = plot_contours_all([X],['rbf'],[.1,1.,10.,100.],True,'6e_rbf_plots',y,True)

#gives perfect score on training set for C = 10, 100.

0.666666666667
0.833333333333
1.0
1.0
